In [4]:
from collections import Counter
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt

In [5]:
import os

if os.path.exists('dataset_eskov.txt'):
    print("YES")


YES


In [6]:
with open('dataset_eskov.txt', encoding="utf8") as f:
    lines = f.readlines()

In [7]:
lines[1]

'Все началось с того, что Дмитрий, прогуливаясь по смотровой площадке, поскользнулся и сорвался с высоты. Он не кричал, не пытался цепляться — он будто смирился с неизбежным. Как рассказывают очевидцы, он падал пластом, словно огромный лист бумаги, распластавшись в воздухе. Сила удара о землю должна была быть разрушительной, но произошло нечто поразительное.\n'

In [8]:
def preprocess(line):
    return ' '.join(w.lower() for w in (''.join(ch for ch in word if ch.isalpha()) for word in line.split()) if w)

In [9]:
preprocess(lines[1])

'все началось с того что дмитрий прогуливаясь по смотровой площадке поскользнулся и сорвался с высоты он не кричал не пытался цепляться он будто смирился с неизбежным как рассказывают очевидцы он падал пластом словно огромный лист бумаги распластавшись в воздухе сила удара о землю должна была быть разрушительной но произошло нечто поразительное'

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        lines,
    ):
        self.lines = lines
        self.pad_token = '<PAD>'
        self.bos_token = '<BOS>'
        self.eos_token = '<EOS>'
        self.uniq_words = [self.pad_token, self.bos_token, self.eos_token] + self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.pad_token_id = self.word_to_index['<PAD>']
        self.bos_token_id = self.word_to_index['<BOS>']
        self.eos_token_id = self.word_to_index['<EOS>']

        self.tokenized = [[self.word_to_index[w] for w in line.split()] for line in self.lines]

    def get_uniq_words(self):
        word_counts = Counter(word for line in self.lines for word in line.split())
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        return (
            torch.LongTensor([self.bos_token_id] + self.tokenized[index]),
            torch.LongTensor(self.tokenized[index] + [self.eos_token_id]),
        )

In [11]:
dataset = Dataset([preprocess(line) for line in lines])

In [12]:
dataset.word_to_index

{'<PAD>': 0,
 '<BOS>': 1,
 '<EOS>': 2,
 'что': 3,
 'на': 4,
 'он': 5,
 'дмитрий': 6,
 'и': 7,
 'с': 8,
 'его': 9,
 'не': 10,
 'по': 11,
 'стало': 12,
 'который': 13,
 'в': 14,
 'все': 15,
 'как': 16,
 'очевидцы': 17,
 'но': 18,
 'асфальт': 19,
 'бок': 20,
 'след': 21,
 'говорят': 22,
 'однажды': 23,
 'еськов': 24,
 'известный': 25,
 'своей': 26,
 'неуемной': 27,
 'тягой': 28,
 'к': 29,
 'высоте': 30,
 'приключениям': 31,
 'оказался': 32,
 'вершине': 33,
 'знаменитого': 34,
 'эмпайрстейтбилдинга': 35,
 'легенде': 36,
 'невероятное': 37,
 'падение': 38,
 'городским': 39,
 'мифом': 40,
 'передают': 41,
 'из': 42,
 'уст': 43,
 'уста': 44,
 'началось': 45,
 'того': 46,
 'прогуливаясь': 47,
 'смотровой': 48,
 'площадке': 49,
 'поскользнулся': 50,
 'сорвался': 51,
 'высоты': 52,
 'кричал': 53,
 'пытался': 54,
 'цепляться': 55,
 'будто': 56,
 'смирился': 57,
 'неизбежным': 58,
 'рассказывают': 59,
 'падал': 60,
 'пластом': 61,
 'словно': 62,
 'огромный': 63,
 'лист': 64,
 'бумаги': 65,
 'распл

In [13]:
word_to_index_list = list(dataset.word_to_index.items())
word_to_index_list[:10]

[('<PAD>', 0),
 ('<BOS>', 1),
 ('<EOS>', 2),
 ('что', 3),
 ('на', 4),
 ('он', 5),
 ('дмитрий', 6),
 ('и', 7),
 ('с', 8),
 ('его', 9)]

In [14]:
len(dataset.word_to_index)

144

In [15]:
dataset[0]

(tensor([ 1, 22,  3, 23,  6, 24, 25, 26, 27, 28, 29, 30,  7, 31, 32,  4, 33, 34,
         35, 11, 36,  9, 37, 38, 12, 39, 40, 13, 41, 42, 43, 14, 44]),
 tensor([22,  3, 23,  6, 24, 25, 26, 27, 28, 29, 30,  7, 31, 32,  4, 33, 34, 35,
         11, 36,  9, 37, 38, 12, 39, 40, 13, 41, 42, 43, 14, 44,  2]))

In [16]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=dataset.pad_token_id)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=dataset.pad_token_id)

    return xx_pad, yy_pad, x_lens, y_lens

In [17]:
dataloader = DataLoader(dataset, batch_size=512, collate_fn=pad_collate, shuffle=True)

In [18]:
class Model(nn.Module):
    def __init__(self, vocab_len):
        super(Model, self).__init__()
        self.hidden_size = 256
        self.embedding_dim = 256
        self.num_layers = 3

        vocab_len = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=vocab_len,
            embedding_dim=self.embedding_dim,
            padding_idx=0,
        )
        self.rnn = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            dropout=0.2,
            batch_first=True,
        )
        self.fc = nn.Linear(self.hidden_size, vocab_len)

    def forward(self, x, lens=None, prev_state=None):
        embed = self.embedding(x)
        if lens is None:
            output, state = self.rnn(embed, prev_state)
        else:
            embed_packed = pack_padded_sequence(embed, lens, batch_first=True, enforce_sorted=False)
            output_packed, state = self.rnn(embed_packed, prev_state)
            output, _ = pad_packed_sequence(output_packed, batch_first=True)
        logits = self.fc(output)
        return logits, state

In [19]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Model(len(dataset.uniq_words)).to(DEVICE).train()

In [20]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

1652880

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index=dataset.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [22]:
def train(model, dataloader, criterion, optimizer, epochs):
    losses = []
    model.train()

    for epoch in range(epochs):
        for x, y, x_lens, y_lens in dataloader:
            optimizer.zero_grad()

            y_pred, _ = model(x.to(DEVICE), x_lens)
            loss = criterion(y_pred.transpose(1, 2), y.to(DEVICE))

            loss.backward()
            optimizer.step()

        losses.append(loss.item())
        if epoch % 10 == 0:
            print({ 'epoch': epoch, 'loss': losses[-1] })

    return losses

In [23]:
loss_history = train(model, dataloader, criterion, optimizer, epochs=150)

{'epoch': 0, 'loss': 4.973462104797363}
{'epoch': 10, 'loss': 2.037584066390991}
{'epoch': 20, 'loss': 1.0881531238555908}
{'epoch': 30, 'loss': 0.7061669230461121}
{'epoch': 40, 'loss': 0.43740835785865784}
{'epoch': 50, 'loss': 0.27588367462158203}
{'epoch': 60, 'loss': 0.17252464592456818}
{'epoch': 70, 'loss': 0.11068646609783173}
{'epoch': 80, 'loss': 0.0828523337841034}
{'epoch': 90, 'loss': 0.06817938387393951}
{'epoch': 100, 'loss': 0.056255921721458435}
{'epoch': 110, 'loss': 0.05193513631820679}
{'epoch': 120, 'loss': 0.04788191244006157}
{'epoch': 130, 'loss': 0.04722565785050392}
{'epoch': 140, 'loss': 0.04140022024512291}


In [24]:
def tokenize(value):
    return [dataset.bos_token_id]+[dataset.word_to_index[word.lower()] for word in value.split()]


def decode(token_ids):
    return ' '.join(dataset.index_to_word[token_id] for token_id in token_ids)


@torch.no_grad()
def generate(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_argmax = logits[0, -1].argmax()
        response.append(token_argmax.item())
        if response[-1] == dataset.eos_token_id:
            break
        model_input = token_argmax.view(1, 1)

    return decode(prompt_tokens + response)

In [25]:
generate('еськов')

'<BOS> еськов началось с того что дмитрий прогуливаясь по смотровой площадке поскользнулся и сорвался с высоты он не кричал не пытался цепляться'

In [26]:
@torch.no_grad()
def sample(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_probs = F.softmax(logits[0, -1], dim=-1).cpu().numpy()
        sampled_token = np.random.choice(len(token_probs), p=token_probs)
        response.append(sampled_token)
        if response[-1] == dataset.eos_token_id:
            break
        model_input = torch.LongTensor([[sampled_token]]).to(DEVICE)

    return decode(prompt_tokens + response)

In [27]:
sample('еськов')

'<BOS> еськов началось с того что дмитрий прогуливаясь по смотровой площадке поскользнулся и сорвался с высоты он не кричал не пытался цепляться'

In [28]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

tokenizer = T5Tokenizer.from_pretrained("sberbank-ai/ruT5-large")

In [29]:
len(dataset.uniq_words)

144

In [30]:
len(set(
    x for tokens in (
        tokenizer(
            preprocess(line),
            add_special_tokens=False,
        )['input_ids'] for line in lines
    ) for x in tokens
))

177